In [1]:
from gurobipy import*

In [2]:
Facility, Capacity = multidict({1:28, 2:20, 3:24, 4:24})
Facility, Fixed_cost = multidict({1:23, 2:12, 3:18, 4:32})
Customer, Demand = multidict({1:22, 2:14, 3:26})
# (Facility, customer)
Transport_cost = {
    (1,1):10, (1,2):24, (1,3):36,
    (2,1):20, (2,2):16, (2,3):14,
    (3,1):30, (3,2):22, (3,3):12,
    (4,1):40, (4,2):30, (4,3):8
}

In [3]:
model = Model("Facility")
x={}
y={}

Academic license - for non-commercial use only - expires 2022-08-16
Using license file /Users/kounuki/gurobi.lic


In [5]:
for f in Facility:
    y[f] = model.addVar(vtype="B",name = "y(%s)"%f)
    for c in Customer:
        x[f,c] = model.addVar(vtype="C",name = "x(%s,%s)"%(f,c))
model.update()

In [6]:
for f in Facility:
    model.addConstr(quicksum(x[f,c] for c in Customer) <= Capacity[f]*y[f],name = "supply constraint")

for c in Customer:
    model.addConstr(quicksum(x[f,c] for f in Facility) >= Demand[c], name = "demand constraint")

In [10]:
model.setObjective(quicksum(Fixed_cost[f]*y[f] for f in Facility)+quicksum(Transport_cost[f,c]*x[f,c] for (f,c) in x))
model.optimize()
print("optimal value",model.objVal)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 32 columns and 28 nonzeros
Model fingerprint: 0x46be4640
Variable types: 24 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [8e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 3e+01]

MIP start from previous solve produced solution with objective 731 (0.01s)
Loaded MIP start from previous solve with objective 731

Presolve removed 0 rows and 16 columns
Presolve time: 0.00s
Presolved: 7 rows, 16 columns, 28 nonzeros
Variable types: 12 continuous, 4 integer (4 binary)

Root relaxation: objective 7.199714e+02, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0       731.00000  731.00000  0.00

In [13]:
EPS = 1.e-6
for (f,c) in x:
    if x[f,c].X > EPS:
        print("X[%s,%s] = %s"%(f,c,x[f,c].X))
for f in Facility:
    if y[f].X > EPS:
        print("y[%s]"%f)

X[1,1] = 22.0
X[2,2] = 14.0
X[2,3] = 2.0
X[4,3] = 24.0
y[1]
y[2]
y[4]
